# Introduction

Bienvenue dans ce notebook où nous allons explorer un aspect fascinant de l'apprentissage profond - la génération automatique de texte. L'objectif de ce projet est d'entraîner un modèle de réseau de neurones à générer des textes qui ressemblent à ceux de William Shakespeare, l'un des écrivains les plus emblématiques de l'histoire.

Pour ce faire, nous utiliserons une architecture de réseau neuronal appelée Long Short-Term Memory (LSTM). Les LSTM sont une sous-catégorie spéciale de réseaux de neurones récurrents (RNN) qui sont particulièrement efficaces pour apprendre de longues séquences de données, comme des phrases ou des paragraphes, et donc très appropriées pour des tâches comme la génération de texte.

Pour entraîner notre modèle, nous utiliserons un corpus de textes de Shakespeare. Ce corpus servira de base d'apprentissage à notre LSTM pour lui permettre d'apprendre le "style" d'écriture de Shakespeare. Une fois entraîné, le modèle sera capable de générer de nouvelles phrases en s'inspirant de ce qu'il a appris de ce corpus.

Il est important de noter que le texte généré par notre modèle ne sera pas un véritable Shakespeare, il n'aura pas le même niveau de profondeur ou de signification. Cependant, il devrait présenter des similitudes stylistiques, comme des tournures de phrase similaires, le vocabulaire et la structure grammaticale que l'on trouve couramment dans les œuvres de Shakespeare.

Dans ce notebook, nous allons :

- Préparer et prétraiter notre corpus de textes.
- Construire notre modèle LSTM à l'aide de Keras, une bibliothèque Python populaire pour l'apprentissage profond.
- Entraîner ce modèle sur notre corpus.
- Générer de nouveaux textes à partir du modèle entraîné.

C'est un voyage passionnant à la croisée de la littérature et de l'intelligence artificielle. Alors, allons-y et voyons ce que notre machine peut créer !

# Préparation et prétraitement des données

Dans ce bloc de code, nous commençons par importer les bibliothèques nécessaires pour notre projet. Cela comprend `random`, `numpy` et plusieurs sous-bibliothèques de `tensorflow` dont nous avons besoin pour construire et entraîner notre modèle LSTM.

Ensuite, nous utilisons la fonction `get_file` de `tensorflow.keras.utils` pour télécharger le fichier de texte 'shakespeare.txt' qui contient notre corpus. Ce fichier est lu, décodé en 'utf-8', converti en minuscules pour normaliser le texte et nous remplaçons les retours à la ligne par des espaces pour faciliter le traitement ultérieur.

Après cela, nous préparons plusieurs structures de données qui nous aideront à transformer notre texte en entrées utilisables pour notre modèle LSTM :

- `characters` : une liste de tous les caractères uniques dans notre texte, triés par ordre alphabétique. Cela servira à construire notre "vocabulaire" pour le modèle.
- `char_to_index` et `index_to_char` : deux dictionnaires qui nous permettent de convertir entre des caractères et des indices numériques. Cela est nécessaire car notre modèle LSTM ne traite pas directement les caractères, mais les indices numériques qui les représentent.

Nous définissons également deux hyperparamètres pour la préparation de nos données : `SEQ_LENGTH` et `STEP_SIZE`. `SEQ_LENGTH` détermine la longueur de chaque séquence d'entrée pour notre modèle, et `STEP_SIZE` détermine l'espacement entre ces séquences. En d'autres termes, pour chaque pas de `STEP_SIZE` caractères dans notre texte, nous préparons une séquence d'entrée de longueur `SEQ_LENGTH`.

Ensuite, nous préparons nos entrées et nos cibles pour le modèle LSTM. Les entrées sont des séquences de `SEQ_LENGTH` caractères à partir de notre texte, et les cibles sont les caractères qui suivent directement ces séquences.

Finalement, nous transformons ces séquences et cibles en représentations "one-hot". C'est une technique courante en apprentissage profond pour gérer les données catégorielles, comme nos caractères. Chaque caractère est représenté par un vecteur de la longueur de notre "vocabulaire", où toutes les positions sont zéro, sauf celle correspondant à ce caractère, qui est un. Ainsi, `x` et `y` sont des matrices tridimensionnelle et bidimensionnelle respectivement, où chaque ligne représente une séquence d'entrée ou une cible, et chaque colonne correspond à un caractère de notre "vocabulaire".

In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
with open(filepath, 'rb') as file:
    text = file.read().decode(encoding='utf-8').lower()

In [4]:
text = text.replace('\n', ' ')

In [5]:
characters = sorted(set(text))
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [6]:
SEQ_LENGTH = 200
STEP_SIZE = 3

sentences = []
next_char = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])
    

In [7]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

# Construction du modèle LSTM et génération de texte

Dans ce bloc de code, nous commençons par construire notre modèle LSTM en utilisant l'API `Sequential` de Keras. Notre modèle comprend trois couches:

1. Une couche LSTM avec 128 unités. C'est la couche qui fera la majeure partie du travail d'apprentissage des séquences de notre texte.
2. Une couche `Dense` (c'est-à-dire une couche de neurones entièrement connectée) avec autant d'unités que nous avons de caractères uniques dans notre texte. Cette couche sert à convertir les sorties de notre couche LSTM en prédictions pour le prochain caractère à générer.
3. Une couche `Activation` qui applique la fonction d'activation "softmax". Cette fonction convertit les scores de la couche Dense en probabilités pour chaque caractère possible.

Après avoir défini l'architecture de notre modèle, nous le compilons avec la fonction de perte `categorical_crossentropy` (adaptée à notre problème de classification multiclasse) et l'optimiseur RMSprop. Ensuite, nous entraînons le modèle sur nos données d'entraînement préparées avec un `batch_size` de 256 pendant 4 `epochs`.

Ensuite, nous définissons deux fonctions pour générer du texte à partir de notre modèle entraîné :

1. `sample` : Cette fonction prend un tableau de prédictions de notre modèle (un score pour chaque caractère possible) et une "température", qui contrôle la diversité des textes générés. Un température plus élevée donne un texte plus aléatoire et diversifié, tandis qu'une température plus basse donne un texte plus prévisible. La fonction utilise ces scores pour générer une distribution de probabilité, puis tire un échantillon de cette distribution pour choisir le prochain caractère.

2. `generate_text` : Cette fonction génère une séquence de texte de longueur donnée en utilisant notre modèle entraîné et la fonction `sample`. Elle commence par choisir aléatoirement une séquence de départ dans notre texte original, puis génère un caractère à la fois, en actualisant la séquence d'entrée à chaque pas.

Finalement, nous générons et affichons une séquence de 20 caractères à partir de notre modèle.

In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

In [9]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.fit(x, y, batch_size=256, epochs=4)

C:\Users\ftiag\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/4
1453/1453 [==============================] - 46s 26ms/step - loss: 1.8373
Epoch 2/4
1453/1453 [==============================] - 37s 26ms/step - loss: 1.5341
Epoch 3/4
1453/1453 [==============================] - 38s 26ms/step - loss: 1.4665
Epoch 4/4
1453/1453 [==============================] - 39s 27ms/step - loss: 1.4316


In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
def generate_text(lenght, temperature):
    # Initialise aléatoirement un numero d'index dans le text
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    # Initialise le texte predit
    generated = ''
    # Initialise la phrase selectionnée dans le texte
    sentence = text[start_index: start_index + SEQ_LENGTH]
    # Ajoute la phrase au texte de sortie
    generated += sentence
    # La boucle parcourt de 0 à "lenght" puis une autre boucle parcour la phrase pour initialisé "x_predictions",
    # ensuite pour tout i on calcule les preds puis on utilise la fonction "sample" qui renvoie l'index du caractère
    # à la fin on prends le caractère puis on l'ajoute à generated et enfin on actualise la phrase.
    for i in range(lenght):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1
        
        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]
        
        generated += next_character
        sentence = sentence[1:] + next_character
    
    return generated

In [13]:
generate_text(20, 0.01)

' thy friends are fled to wait upon thy foes, and crossly to thy good all fortune goes.  henry bolingbroke: bring forth these men. bushy and green, i will not vex your souls-- since presently your soul so the heart to the'